# LJ Potential: scipy.minimize with analytical gradient (jacobian)

I define a function LJ_jac that I can pass into the minimize function in order to speed up the minimization process. This function uses a double for loop to calculate the total force (equal to the gradient of the potential) felt by a central atom by each of its neighboring atoms. Then, it returns the force felt by each atom, or analytical gradient of the LJ potential, for each x,y,z coordinate.

In [578]:
import numpy as np
from numba import jit


@jit
def LJ(r):
    r6 = r**6
    r12 = r6*r6
    return 4*(1/r12 - 1/r6)

@jit
def total_energy(positions):
    """
    Calculate the total energy
    input:
    positions: 3*N array which represents the atomic positions
    output
    E: the total energy
    """
    E = 0
    #N_atom = int(len(positions))
    N_atom = int(len(positions)/3)
    
    #positions = [x0, y0, z0, x1, y1, z1, .....  , x(n-1), y(n-1), z(n-1)]
    for i in range(N_atom-1):
        for j in range(i+1, N_atom):
            pos1 = positions[i*3:(i+1)*3]
            pos2 = positions[j*3:(j+1)*3]
            #pos1 = positions[i]
            #pos2 = positions[j]
            #print('pos1:  ', pos1)
            #print('pos2:  ', pos2)
            #dist = np.abs(pos1-pos2)
            dist = np.linalg.norm(pos1-pos2)
            #print(i,j, dist)
            E += LJ(dist)
    return E

@jit
def init_pos(N, L=5):
    return L*np.random.random_sample((N*3,))

@jit
def pos_r_space(pos):
    pos_r = []
    for i in range(len(pos)//3):
        pos_r.append(np.linalg.norm(pos[i*3:(i+1)*3]))
    return pos_r

"""
Function: Calculate and return the Jacobian for N atoms in the LJ potential.
"""
@jit
def LJ_jac(pos):
    N_atom = int(len(pos)/3) 
    force = np.zeros([N_atom, 3])   # Create force array to hold total force felt by each central atom
                 
    for i in range(N_atom): # First for-loop iterates through each central atom
        pos1 = pos[i*3:(i+1)*3] # Define the position of the central atom
        for j in range(N_atom): # Second for-loop iterates through neighboring atoms
            pos2 = pos[j*3:(j+1)*3]
            if not any(pos1 == pos2): # Ensure we don't compute an atom with itself
                dist_xyz = pos1 - pos2
                dist_r = np.linalg.norm(dist_xyz)
                r14 = np.power(dist_r, 14)
                r8 = np.power(dist_r, 8)
                next_r = np.dot((-48/r14 + 24/r8), dist_xyz)
                #print('next_r: ', next_r)
                force[i] += next_r
    
    #force = np.reshape(force, [1, len(pos)])
    return force.flatten() #reshape force array into 1 X (N-atom*3) array

In [579]:
from scipy.optimize import minimize
import time


x_values = []
#N_attempts = 1
N_atoms = [4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
Energies = [-6.000000, -9.103852, -12.712062, -16.505384, -19.821489, -24.113360, -28.422532, -32.765970, 
-37.967600, -44.326801, -47.845157, -52.322627, -56.815742, -61.317995, -66.530949, 
-72.659782, -77.177043]
    

"""
minimize 

Method: L-BFGS-B

"""
globalstart = time.time()
for n in N_atoms:
    
    fun_values = []
    f_values = 0
    count = 0
    start = time.time()
    while (f_values-Energies[n-4]>0.8 and count < 100):
        pos = init_pos(n)
        #pos_r = pos_r_space(pos)
        res = minimize(total_energy, pos, jac=LJ_jac, method='L-BFGS-B', tol=1e-4)
        f_values = res.fun
        fun_values.append(res.fun)
        #x_values.append(res.x)
        count +=1
        print('\r Step: {:d} out of {:d}; value: {:.4f} Time {:.2f} s'.format(count, 200 , res.fun, time.time()-start), flush=True, end='')
   # if i%10==0:
       # print('Step: ', i, '  values:', res.fun)
    if (f_values-Energies[n-4]<= 0.8):
        print('\n The global minimum for N =  ', n,  f_values)
    else:
        print('\n The global minimum for N = ', n, 'is', min(fun_values))
    
print('Total time taken: ', time.time()-globalstart, 's')

<ipython-input-578-f4d5bbd6f38b>:53: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "LJ_jac" failed type inference due to: Untyped global name 'any': cannot determine Numba type of <class 'builtin_function_or_method'>

File "<ipython-input-578-f4d5bbd6f38b>", line 62:
def LJ_jac(pos):
    <source elided>
            pos2 = pos[j*3:(j+1)*3]
            if not any(pos1 == pos2): # Ensure we don't compute an atom with itself
            ^

  @jit
<ipython-input-578-f4d5bbd6f38b>:53: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "LJ_jac" failed type inference due to: cannot determine Numba type of <class 'numba.dispatcher.LiftedLoop'>

File "<ipython-input-578-f4d5bbd6f38b>", line 58:
def LJ_jac(pos):
    <source elided>
                 
    for i in range(N_atom): # First for-loop iterates through each central atom
    ^

  @jit
C:\Users\durke\OneDrive\Documents\Anaconda3\lib\sit

 Step: 9 out of 200; value: -6.0000 Time 4.63 s
 The global minimum for N =   4 -5.999985899895553
 Step: 2 out of 200; value: -9.1038 Time 0.20 s
 The global minimum for N =   5 -9.103780166740467
 Step: 9 out of 200; value: -12.3020 Time 0.75 s
 The global minimum for N =   6 -12.301975397186853
 Step: 34 out of 200; value: -15.9348 Time 6.19 s
 The global minimum for N =   7 -15.934761690648019
 Step: 6 out of 200; value: -19.8211 Time 1.45 s
 The global minimum for N =   8 -19.82114461488706
 Step: 21 out of 200; value: -24.1128 Time 8.46 s
 The global minimum for N =   9 -24.11278134947112
 Step: 68 out of 200; value: -28.4220 Time 35.53 s
 The global minimum for N =   10 -28.42204973585571
 Step: 61 out of 200; value: -32.7650 Time 57.80 s
 The global minimum for N =   11 -32.764992892211225
 Step: 24 out of 200; value: -37.9662 Time 22.39 s
 The global minimum for N =   12 -37.96619085242904
 Step: 100 out of 200; value: -35.2411 Time 95.88 s
 The global minimum for N =  13 is -